In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# import os
# try:
#     os.chdir("./drive/My Drive/Кванториум/Большие вызовы")
# except: pass

# Загрузка данных

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter("ignore")

df = pd.read_excel("Sample - Superstore.xls")

In [3]:
# используем праздники

import pandas as pd
holidays = pd.read_csv("holidays.csv")
arr = []
for element in holidays.values:
    year = element[0]
    for month, days in enumerate(element[1:13], start=1):
        days = days.replace("*", "").replace("+", "").split(",")
        for day in days:
            arr.append([pd.to_datetime(f"{year}-{month}-{day}"), "Выходные"])

df_holidays = pd.DataFrame(arr, columns=["ds", "holiday"])
df_holidays.head()

,ds,holiday
0,1999-01-01,Выходные
1,1999-01-02,Выходные
2,1999-01-03,Выходные
3,1999-01-04,Выходные
4,1999-01-06,Выходные


# Обработка последовательностей

In [4]:
# переводим дату в числа
def date_to_data(frame, holidays=None,
                 drop=False, name="date", suffix=""):
    assert name in frame, "Нужна колонка с датой"
    date = frame[name].dt

    info = pd.concat([date.days_in_month,
                        date.month,
                        date.quarter], axis=1)

    info.columns = ["days_in_month", "month", "quarter"]

    temp = pd.concat([date.isocalendar(),
                        info], axis=1)

    temp.columns = temp.columns + suffix
    frame = pd.concat([frame, temp], axis=1)
    if "holiday" not in frame:
        frame = frame.merge(holidays, left_on=name, right_on="ds", how="left") \
            .drop(["ds"], axis=1)
        frame["holiday"] = frame["holiday"].apply(lambda x: 1 if type(x) == str else 0)
    if drop:
        return frame.drop([name], axis=1)
    return frame

In [5]:
df_new = date_to_data(df, df_holidays, name="Order Date",
                      suffix="_Order", drop=False)

df_new = date_to_data(df_new, df_holidays, name="Ship Date",
                      suffix="_Ship",  drop=False)

In [6]:
from sklearn.preprocessing import LabelEncoder

# закодируем строки
def cleaning_frame(df):
    for column in df.drop(["Sales"], axis=1):
        try:
            df[column] = df[column].astype(np.int32)
        except:
            df[column] = df[column].astype(np.str)

    data_categorical = df.select_dtypes(include=['object'])
    data_numeric = df.select_dtypes(exclude=['object'])

    for column in data_categorical:
        data_categorical[column] = LabelEncoder() \
            .fit_transform(data_categorical[column])

    df = pd.concat([data_numeric, data_categorical], axis=1)
    return df

In [7]:
# сортируем по дате
df_new = cleaning_frame(df_new).sort_values(by="Order Date")

In [8]:
from sklearn.model_selection import train_test_split

def train_test_val_split(X, y,
                         test_size=0.3, val_size=0.3,
                         random_state=42, shuffle=True):
    """
    separator for train, test, validation
    """
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                        train_size=1 - (test_size + val_size),
                                                        random_state=random_state, shuffle=shuffle)
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test,
                                                    test_size=test_size / (test_size + val_size),
                                                    random_state=random_state, shuffle=shuffle)
    return X_train, y_train, X_test, y_test, X_val, y_val

In [9]:
X, y = df_new.drop(["Sales"], axis=1), df_new["Sales"]
X_train, y_train, X_test, y_test, X_val, y_val = train_test_val_split(X, y,
                                                                      test_size=0.2,
                                                                      val_size=0.1,
                                                                      shuffle=False)

In [10]:
from tqdm import tqdm

def merge_frames(source, names: list, target: str, bar=False):
    for name in tqdm(names, disable=bar):
        means = source.groupby(by=name).aggregate(["median", "min", "mean",
                                          "max", "sum"])[target]
        means.columns = means.columns + f"_{name}"
        source = source.merge(means, on=name, how="left")
    return source

def prepare(X, y, names):
    X["y"] = y
    out = merge_frames(X, names, "y")
    return out.drop(["y"], axis=1), out["y"]

In [11]:
# столбцы по которым считаются статистические метрики
data_agg = ["Customer ID", "Product ID", ["Customer ID", "Product ID"], ["Postal Code", "Product ID"],
            "Ship Mode", "Category", "Sub-Category", ["Category", "Sub-Category"], "Order Date", "Ship Date",
            "Region", "Postal Code", "State", "City", "Segment", "Quantity",
            ["Region", "State", "City"], ["Ship Mode", "Segment"], "Discount", "Profit",
            ["Ship Mode", "Segment", "Quantity"], ["Region", "State"], ["Region", "State", "City", "Postal Code"],
            ["Region", "City"], ["Region", "State", "City", "Quantity"],
            ["Customer ID", "Product ID", "Postal Code"], ["Discount", "Profit"],
            ["Discount", "Profit", "Category", "Sub-Category"], ["Discount", "Profit", "Category"],
            ["Discount", "Profit", "Sub-Category"], "week_Order", "day_Order", ["week_Order", "day_Order"],
            "days_in_month_Order", ["week_Order", "day_Order", "days_in_month_Order"],
            ["day_Order", "days_in_month_Order"], ["week_Order", "days_in_month_Order"]]

In [12]:
# независимо расчитываем функции
X_train, y_train = prepare(X_train, y_train, data_agg)
X_test, y_test = prepare(X_test, y_test, data_agg)
X_val, y_val = prepare(X_val, y_val, data_agg)

100%|██████████| 37/37 [00:12<00:00,  2.93it/s]


In [13]:
# колонки, которые нам не нужны
trash = ["Country", "Row ID",
         "Order ID", "Customer ID",
         "Product ID", "Order Date", "Ship Date"]

X_train.drop(trash, axis=1, inplace=True)
X_test.drop(trash, axis=1, inplace=True)
X_val.drop(trash, axis=1, inplace=True)

# Обучение модели

In [14]:
from sklearn.linear_model import LassoCV

model = LassoCV(max_iter=100,
                normalize=True,
                n_jobs=-1)

model.fit(X_train,
          y_train)

LassoCV(alphas=None, copy_X=True, cv=None, eps=0.001, fit_intercept=True,
        max_iter=100, n_alphas=100, n_jobs=-1, normalize=True, positive=False,
        precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
        verbose=False)

# Предсказание на будущее и оценка алгоритма

In [15]:
from sklearn.metrics import (r2_score,
                             mean_squared_error,
                             mean_absolute_error)

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def get_metrics(y_true, y_pred):
    print(f"R2: {r2_score(y_true, y_pred)}")
    print(f"MAE: {mean_absolute_error(y_true, y_pred)}")
    print(f"MSE: {mean_squared_error(y_true, y_pred)}")
    print(f"MAPE: {mean_absolute_percentage_error(y_true, y_pred)}")

## Валидация

In [16]:
predictions = model.predict(X_val)
get_metrics(y_val, predictions)

R2: 0.9999990474956869
MAE: 0.2594896216133304
MSE: 0.3373448648198216
MAPE: 1.241178163637693


## Тестирование

In [17]:
predictions = model.predict(X_test)
get_metrics(y_test, predictions)

R2: 0.9991226422027965
MAE: 0.9455433378781429
MSE: 300.79825255945406
MAPE: 1.3085624399677345
